In [ ]:
!pip install -U datasets beautifulsoup4 pillow transformers accelerate bitsandbytes transformers peft


[notice] A new release of pip is available: 23.3.1 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


# Part 1: Preprocess Mind2Web

In [ ]:
import os

# Force Hugging Face to download models to the large /workspace volume
# instead of the tiny system disk.
os.environ["HF_HOME"] = "/workspace/hf_cache"

print(f"Hugging Face Cache is now set to: {os.environ['HF_HOME']}")

Hugging Face Cache is now set to: /workspace/hf_cache


In [ ]:
import datasets
from PIL import Image

# Load the correct, flattened multimodal dataset from Hugging Face
# Using streaming=True is still recommended to save disk space
multimodal_dataset = datasets.load_dataset("osunlp/Multimodal-Mind2Web", split="train", streaming=True)

Resolving data files:   0%|          | 0/27 [00:00<?, ?it/s]

In [ ]:
import datasets
from PIL import Image

# Get the first training example (which is a single action) to inspect it
first_example = next(iter(multimodal_dataset))

# --- Print out the key fields to see the new data structure ---
print("--- First Training Example (Single Action) ---")
print(f"Goal: {first_example['confirmed_task']}")

# This is the specific action for this row
print(f"\nTarget Action Representation: {first_example['target_action_reprs']}")

# The HTML is now a direct field of the example
print("\nCleaned HTML Snippet (first 500 chars):")
print(first_example['cleaned_html'][:500])

# The 'screenshot' field will be loaded as a Pillow Image object directly
print("\nScreenshot object:")
screenshot_image = first_example['screenshot']
print(screenshot_image)
print(f"Image mode: {screenshot_image.mode}, Image size: {screenshot_image.size}")

# You can display the image in Colab by having `screenshot_image` as the last line
# screenshot_image

--- First Training Example (Single Action) ---
Goal: rent a car in Brooklyn - Central, NY on from April 9 to April 15.

Target Action Representation: [heading]  CAR -> CLICK

Cleaned HTML Snippet (first 500 chars):
<html backend_node_id="208">
  <body backend_node_id="500">
    <div backend_node_id="1054">
      <div backend_node_id="1055">
        <div backend_node_id="1056">
          <div backend_node_id="1057">
            <div backend_node_id="1060">
              <div backend_node_id="1064">
                <h1 backend_node_id="1065">
                  <text backend_node_id="1066">Welcome to United.com</text>
                </h1>
                <a backend_node_id="1067">
                  <text bac

Screenshot object:
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1280x5429 at 0x7AAAD53ABF70>
Image mode: RGB, Image size: (1280, 5429)


In [ ]:
from bs4 import BeautifulSoup
import json
import re

## Data ended up being processed in groundhog-data-processing.ipynb

# Part 2: Load Preprocessed Mind2Web

In [ ]:
from google.colab import drive

# This command will initiate the mounting process.
drive.mount('/content/drive')

In [ ]:
import json

# --- Access the file from your mounted Google Drive ---

# The path to your file is now fixed and reliable.
# 'My Drive' is the standard name for the root of your Google Drive.
FILENAME = "/workspace/mind2web_processed_train.jsonl"

NUM_LINES_TO_CHECK = 3

print(f"--- Checking the first {NUM_LINES_TO_CHECK} lines of the file from Google Drive ---")

with open(FILENAME, "r") as f:
    for i, line in enumerate(f):
        if i >= NUM_LINES_TO_CHECK:
            break

        # Parse the JSON string from the line into a Python dictionary
        data = json.loads(line)

        print(f"\n--- Example {i+1} ---")
        print("Prompt Snippet:")
        # Print the first 200 characters of the prompt
        print(data['prompt'] + "...")

        print("\nLabel:")
        # The label is a string, so we can parse it again to pretty-print
        pretty_label = json.loads(data['label'])
        print(json.dumps(pretty_label, indent=2))

--- Checking the first 3 lines of the file from Google Drive ---

--- Example 1 ---
Prompt Snippet:
You are a web agent. Analyze the screenshot and the list of elements.
The element list is formatted as: [ID] <Tag> Text (Attributes).
If the target element is not in the list, select ID 0.
Your task is to select the correct Element ID to perform the action on.

TASK: rent a car in Brooklyn - Central, NY on from April 9 to April 15.

ELEMENTS:
[0] <option> Target element is not in this list
[-] <h1> Welcome to United.com
[1067] <a> Skip to book
[1085] <li> English - United States$
[1088] <button> English - United States$
[1096] <button>
[1100] <li> Search
[1101] <a> Search
[1112] <button> Hi, James 0 miles
[1129] <button>
[1133] <li> Menu
[1134] <button> Menu
[1145] <a> BOOK (role='tab')
[1148] <a> MY TRIPS (role='tab')
[1151] <a> TRAVEL INFO (role='tab')
[1154] <a> MILEAGEPLUS PROGRAM (role='tab')
[1157] <a> DEALS (role='tab')
[1163] <button>
[1168] <button>
[1173] <button>
[1178] <butto

## Screenshot saving

In [ ]:
#saving cropped screenshots

from PIL import Image
import os
from tqdm.auto import tqdm

# IMPORTANT: Allow loading large images to prevent "DecompressionBombError"
Image.MAX_IMAGE_PIXELS = None

SAVE_DIR = "/content/drive/My Drive/m2w_i"
os.makedirs(SAVE_DIR, exist_ok=True)

# 1. FIXED WIDTH: 1024 is standard for 7B VLMs.
# It keeps text readable but is much cheaper than 1280.
TARGET_WIDTH = 1024

# 2. MAX HEIGHT: We hard-crop anything below this.
# 1280 height usually covers the "fold" and slightly more.
MAX_HEIGHT = 1280

print(f"Saving screenshots to: {SAVE_DIR}")
print(f"Strategy: Resize width to {TARGET_WIDTH}px, then CROP height to max {MAX_HEIGHT}px.")

for example in tqdm(multimodal_dataset):
    try:
        annotation_id = example["annotation_id"]
        img = example["screenshot"]  # PIL image

        # --- NEW RESIZING LOGIC ---

        # 1. Calculate new height to preserve aspect ratio
        w_percent = (TARGET_WIDTH / float(img.size[0]))
        h_size = int((float(img.size[1]) * float(w_percent)))

        # 2. Resize the image (High Quality)
        # We enforce the width, letting height float
        img_resized = img.resize((TARGET_WIDTH, h_size), Image.LANCZOS)

        # 3. Smart Crop (Viewport Simulation)
        # If the resulting image is too tall, we chop off the bottom.
        # This keeps the header/nav/main content crisp.
        if h_size > MAX_HEIGHT:
            # Crop box: (left, upper, right, lower)
            img_resized = img_resized.crop((0, 0, TARGET_WIDTH, MAX_HEIGHT))

        # --- END OF NEW CODE ---

        # Save as JPEG (optimized)
        path = os.path.join(SAVE_DIR, f"{annotation_id}.jpeg")
        img_resized.convert('RGB').save(path, quality=85)

    except Exception as e:
        print(f"Skipping {annotation_id} because of error: {e}")

print("Done saving all processed images.")

In [ ]:
import shutil
print("Zipping images on Drive... this takes a minute...")
shutil.make_archive("/content/drive/My Drive/m2w_images", 'zip', "/content/drive/My Drive/m2w_i")
print("Done! Created m2w_images.zip on your Drive.")

### Download Screenshots to local

In [ ]:
import os

# Path to local fast disk
LOCAL_IMAGE_FOLDER = "/content/m2w_i_local"
ZIP_PATH = "/content/drive/My Drive/m2w_images.zip"

if not os.path.exists(LOCAL_IMAGE_FOLDER):
    print("🚀 Copying ZIP from Drive...")
    !cp "{ZIP_PATH}" /content/temp_images.zip

    print("📂 Unzipping to local disk...")
    !unzip -q /content/temp_images.zip -d "{LOCAL_IMAGE_FOLDER}"

    print(f"✅ Ready! Images are at {LOCAL_IMAGE_FOLDER}")
else:
    print("✅ Images already loaded.")

# ✅ Part 3: FineTune Qwen2.5-VL 7B via LoRA (NEW) ✅

## Dataset Builder

In [ ]:
from transformers import AutoModelForCausalLM, AutoProcessor
import torch
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import os

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'Could not load this library: /usr/local/lib/python3.10/dist-packages/torchvision/image.so'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [ ]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

In [ ]:
from transformers import AutoProcessor, AutoModelForImageTextToText, BitsAndBytesConfig, Qwen2_5_VLForConditionalGeneration

model_id = "Qwen/Qwen2.5-VL-7B-Instruct"

processor = AutoProcessor.from_pretrained(
    model_id,
    trust_remote_code=True,
    use_fast=False,
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-7B-Instruct",
    device_map="auto",
    quantization_config=bnb_config
)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
prepare_model_for_kbit_training(model)

lora = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora)

# The standard model.enable_input_require_grads() is failing silently.
# We do it manually here.
def make_inputs_require_grad(module):
    found = False
    for name, child in module.named_modules():
        if isinstance(child, torch.nn.Embedding):
            # Force gradients on the embedding layer
            child.weight.requires_grad_(True)
            print(f"✅ FORCED GRADIENTS: {name}")
            found = True
    if not found:
        print("❌ WARNING: Could not find embedding layer to unlock!")

make_inputs_require_grad(model)
model.gradient_checkpointing_enable()
model.print_trainable_parameters()
model.config.use_cache = False

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

✅ FORCED GRADIENTS: base_model.model.model.language_model.embed_tokens
trainable params: 568,792,064 || all params: 8,315,961,344 || trainable%: 6.8398


In [ ]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image, UnidentifiedImageError
import os, json

In [ ]:
def safe_open(path):
    try:
        return Image.open(path).convert("RGB")
    except Exception:
        return None

In [ ]:
class Mind2WebQwenVLDataset(Dataset):
    def __init__(self, jsonl_path, image_folder, processor, skip_checks=False):
        self.processor = processor
        self.image_folder = image_folder
        self.skip_checks = skip_checks

        # Track removed samples (only used if skip_checks=False)
        self.removed_ids = []

        # Load all JSON samples
        with open(jsonl_path, "r") as f:
            raw = [json.loads(line) for line in f]

        self.samples = []
        removed = 0

        for s in raw:
            annotation_id = s["annotation_id"]

            # If skip_checks is True, just store the sample and continue
            if self.skip_checks:
                s["_image_path"] = os.path.join(image_folder, f"{annotation_id}.jpeg")
                self.samples.append(s)
                continue

            # Otherwise, check image existence/corruption
            img_path = None
            for ext in ["png", "jpg", "jpeg", "webp"]:
                p = os.path.join(image_folder, f"{annotation_id}.{ext}")
                if os.path.isfile(p):
                    img_path = p
                    break

            if img_path is None:
                removed += 1
                self.removed_ids.append(annotation_id)
                continue

            img = safe_open(img_path)
            if img is None:
                removed += 1
                self.removed_ids.append(annotation_id)
                continue

            s["_image_path"] = img_path
            self.samples.append(s)
            print(f"Sample Image Path: {self.samples[0]['_image_path']}")

        if not self.skip_checks:
            print(f"Filtered dataset: {len(raw)} → {len(self.samples)} ({removed} removed)")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]

        # If skip_checks=True, reconstruct the image path assuming cleaned dataset
        if self.skip_checks:
            # adjust extension if needed
            img_path = os.path.join(self.image_folder, f"{sample['annotation_id']}.jpeg")
        else:
            img_path = sample["_image_path"]

        try:
            with Image.open(img_path) as f:
                img = f.convert("RGB")
        except Exception:
            # Fallback for stability
            img = Image.new("RGB", (1024, 1024), (0, 0, 0))

        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": sample["prompt"]},
                    {"type": "image", "image": img},
                ],
            },
            {
                "role": "assistant",
                "content": [
                    {"type": "text", "text": sample["label"]},
                ],
            },
        ]

        return messages

In [ ]:
def qwen_collate_fn(batch):
    texts = []
    images = []

    for messages in batch:
        # Extract the formatted prompt via the chat template
        txt = processor.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )
        texts.append(txt)

        # Extract images (one per example here)
        img = [c["image"] for c in messages[0]["content"] if c["type"] == "image"]
        images.append(img)

    # Processor handles image + text merging into the multimodal embedding
    out = processor(
        text=texts,
        images=images,
        padding=True,
        return_tensors="pt",
        max_length=8192,
        truncation=True
    )

    out["labels"] = out["input_ids"].clone()
    return out

## ❗For filtering out the corrupted samples and saving to new dataset❗

In [ ]:
dataset = Mind2WebQwenVLDataset(
    jsonl_path="/content/drive/My Drive/mind2web_processed_train.jsonl",
    image_folder="/content/m2w_i_local",
    processor=processor
)

loader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=True,
    collate_fn=qwen_collate_fn
)

In [ ]:
print(dataset.removed_ids)

In [ ]:
for ann_id in dataset.removed_ids:
    for ext in ["png", "jpg", "jpeg", "webp"]:
        path = os.path.join("/content/drive/My Drive/m2w_i", f"{ann_id}.{ext}")
        if os.path.isfile(path):
            os.remove(path)

In [ ]:
import json

removed = set(dataset.removed_ids)
cleaned = []

with open("/content/drive/My Drive/mind2web_processed_train.jsonl") as f:
    for line in f:
        js = json.loads(line)
        if js["annotation_id"] not in removed:
            cleaned.append(js)

with open("/content/drive/My Drive/mind2web_processed_train_clean.jsonl", "w") as f:
    for js in cleaned:
        f.write(json.dumps(js) + "\n")

print("Wrote cleaned JSONL:", len(cleaned))

## ❗ ✅ for loading in the saved dataset

In [ ]:
from torch.utils.data import DataLoader

dataset = Mind2WebQwenVLDataset(
    jsonl_path="/workspace/mind2web_processed_train.jsonl",
    image_folder="/workspace/m2w_i_local",
    processor=processor,
    skip_checks=True
)

loader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=True,
    collate_fn=qwen_collate_fn,
    num_workers=2,
    pin_memory=True,
    persistent_workers=True
)

print(f"Number of examples: {len(dataset)}")

Number of examples: 5743


In [ ]:
import torch
from tqdm import tqdm

#optional: cautionary check for max input sequence token length + max image token length

max_seq_len = 0
max_pixel_dim0 = 0
max_pixel_shape = None

# We'll also store the ID of the problematic samples for inspection
sample_with_max_seq = None
sample_with_max_pixel = None

# Loop through the dataset with a progress bar
for i in tqdm(range(len(dataset)), desc="Analyzing dataset samples"):
    # Get the raw messages for one sample
    messages = dataset[i]

    # Use the collate function to process it, wrapping it in a list to simulate a batch of 1
    batch = qwen_collate_fn([messages])

    # --- Check sequence length ---
    current_seq_len = batch['input_ids'].shape[1]
    if current_seq_len > max_seq_len:
        max_seq_len = current_seq_len
        # Get the original sample's metadata to find the ID
        sample_with_max_seq = dataset.samples[i]['annotation_id']

    # --- Check pixel_values dimensions ---
    current_pixel_shape = batch['pixel_values'].shape
    # The first dimension is usually the one that varies with image size/patches
    current_pixel_dim0 = current_pixel_shape[0]

    if current_pixel_dim0 > max_pixel_dim0:
        max_pixel_dim0 = current_pixel_dim0
        max_pixel_shape = current_pixel_shape
        sample_with_max_pixel = dataset.samples[i]['annotation_id']

# --- Print the results ---
print("\n--- Analysis Complete ---")
print(f"Max sequence length found: {max_seq_len}")
print(f"  - Found in sample ID: {sample_with_max_seq}\n")

print(f"Max pixel_values shape found: {max_pixel_shape}")
print(f"  - Found in sample ID: {sample_with_max_pixel}")

In [ ]:
# Just to sanity check one example
batch = next(iter(loader))
print(batch.keys())  # should contain input_ids, pixel_values, attention_mask, labels
print(batch['pixel_values'].shape)
print(batch['input_ids'].shape)

KeysView({'input_ids': tensor([[151644,   8948,    198,  ...,     92, 151645,    198]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]]), 'pixel_values': tensor([[-0.7704, -0.7704, -0.7704,  ..., -0.2573, -0.2573, -0.2573],
        [-0.7704, -0.7704, -0.7704,  ..., -0.2573, -0.2573, -0.2573],
        [-0.7704, -0.7704, -0.7704,  ..., -0.2289, -0.2289, -0.2289],
        ...,
        [ 1.9303,  1.9303,  1.9303,  ...,  2.1459,  2.1459,  2.1459],
        [ 1.9303,  1.9303,  1.9303,  ...,  2.1459,  2.1459,  2.1459],
        [ 1.9303,  1.9303,  1.9303,  ...,  2.1459,  2.1459,  2.1459]]), 'image_grid_thw': tensor([[ 1, 92, 74]]), 'labels': tensor([[151644,   8948,    198,  ...,     92, 151645,    198]])})
torch.Size([6808, 1176])
torch.Size([1, 5003])


## Training Loop - Final Checks

In [ ]:
import torch
from torch.optim import AdamW
from torch.cuda.amp import autocast
from torch.amp import GradScaler
from tqdm import tqdm
from transformers import get_scheduler
import bitsandbytes as bnb

In [ ]:
EPOCHS = 5
LR = 2e-5
GRADIENT_ACCUM = 8
MAX_GRAD_NORM = 1.0
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ------------------------------
# OPTIMIZER AND SCALER
# ------------------------------
model.train()

optimizer = bnb.optim.AdamW8bit(filter(lambda p: p.requires_grad, model.parameters()), lr=LR, weight_decay=0.0)
scaler = GradScaler('cuda')

In [ ]:
for name, p in model.named_parameters():
    if p.requires_grad:
        print(name, p.device, p.dtype)

base_model.model.model.visual.blocks.0.mlp.gate_proj.lora_A.default.weight cuda:0 torch.float32
base_model.model.model.visual.blocks.0.mlp.gate_proj.lora_B.default.weight cuda:0 torch.float32
base_model.model.model.visual.blocks.0.mlp.up_proj.lora_A.default.weight cuda:0 torch.float32
base_model.model.model.visual.blocks.0.mlp.up_proj.lora_B.default.weight cuda:0 torch.float32
base_model.model.model.visual.blocks.0.mlp.down_proj.lora_A.default.weight cuda:0 torch.float32
base_model.model.model.visual.blocks.0.mlp.down_proj.lora_B.default.weight cuda:0 torch.float32
base_model.model.model.visual.blocks.1.mlp.gate_proj.lora_A.default.weight cuda:0 torch.float32
base_model.model.model.visual.blocks.1.mlp.gate_proj.lora_B.default.weight cuda:0 torch.float32
base_model.model.model.visual.blocks.1.mlp.up_proj.lora_A.default.weight cuda:0 torch.float32
base_model.model.model.visual.blocks.1.mlp.up_proj.lora_B.default.weight cuda:0 torch.float32
base_model.model.model.visual.blocks.1.mlp.down_

In [ ]:
import torch

try:
    # 1. Access the Input Embeddings
    # For Qwen, this is usually model.model.model.embed_tokens or similar
    input_embeddings = model.get_input_embeddings()

    print(f"Checking Layer: {input_embeddings}")

    # 2. Check the Status
    status = input_embeddings.weight.requires_grad
    print(f"\nExample Weight Value: {input_embeddings.weight[0][0].item()}")
    print(f"Gradients Enabled: {status}")

    if status == False:
        print("\n❌ CRITICAL FAIL: Gradients are FALSE. The model cannot learn properly.")
    else:
        print("\n✅ SUCCESS: Gradients are TRUE. The model is fine.")

except Exception as e:
    print(f"Error checking gradients: {e}")

Checking Layer: Embedding(152064, 3584)

Example Weight Value: -0.01708984375
Gradients Enabled: True

✅ SUCCESS: Gradients are TRUE. The model is fine.


In [ ]:
def brute_force_enable_grads(module):
    found = False
    print("Searching for Embedding layer to unlock...")

    # Walk through every single submodule
    for name, child in module.named_modules():
        if isinstance(child, torch.nn.Embedding):
            print(f"📍 Found Embedding Layer at: {name}")

            # FORCE IT: Use the in-place underscore method
            child.weight.requires_grad_(True)

            # Double check
            if child.weight.requires_grad:
                print(f"   ✅ FORCE SUCCESS: {name}.weight.requires_grad is now TRUE")
                found = True
            else:
                print(f"   ❌ FORCE FAILED: {name} is locked.")

    return found

# Execute the force fix
brute_force_enable_grads(model)

# Re-enable checkpointing just in case
model.gradient_checkpointing_enable()

Searching for Embedding layer to unlock...
📍 Found Embedding Layer at: base_model.model.model.language_model.embed_tokens
   ✅ FORCE SUCCESS: base_model.model.model.language_model.embed_tokens.weight.requires_grad is now TRUE


In [ ]:
import numpy as np

# 1. Get a single batch from your loader
print("Testing Data Loader...")
iterator = iter(loader)
messages = next(iterator) # Uses your collation logic implicitly via loader

# 2. Dig into the raw dataset to check the first image path
raw_sample = dataset.samples[0]
img_path = raw_sample["_image_path"]
print(f"\nChecking Path: {img_path}")

if os.path.exists(img_path):
    print("✅ File exists on disk.")
    file_size = os.path.getsize(img_path)
    print(f"   Size: {file_size/1024:.2f} KB")
    if file_size < 1000:
        print("⚠️ WARNING: File is suspiciously small.")
else:
    print("❌ ERROR: File does NOT exist. The model is seeing Black Images.")

# 3. Check what the dataset actually returned
# We act like __getitem__ to see if the try/except block triggered
try:
    img = Image.open(img_path).convert("RGB")
    # check if it's all black
    extrema = img.getextrema()
    if extrema == ((0, 0), (0, 0), (0, 0)):
        print("⚠️ Image is completely BLACK (0,0,0).")
    else:
        print("✅ Image loaded and has content (not pure black).")
except Exception as e:
    print(f"❌ Exception during load: {e}")
    print("   The model is falling back to the Black Image.")

Testing Data Loader...

Checking Path: /workspace/m2w_i_local/401c4e6f-6b0b-47b4-8157-92d7ca468bbc.jpeg
✅ File exists on disk.
   Size: 243.09 KB
✅ Image loaded and has content (not pure black).


## Main Training Loop

In [ ]:
total_training_steps = (len(loader) // GRADIENT_ACCUM + 1) * EPOCHS
scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=int(0.03 * total_training_steps),  # 3% warmup
    num_training_steps=total_training_steps
)

In [ ]:
import os
import torch
import re
from tqdm import tqdm
from safetensors.torch import load_file

checkpoint_dir = "/workspace/qwen_checkpoints"
final_model_dir = "/workspace/mind2web_trained_model"
os.makedirs(checkpoint_dir, exist_ok=True)
os.makedirs(final_model_dir, exist_ok=True)

SAVE_EVERY_N_STEPS = 200

latest_checkpoint_dir = None
global_step = 0
start_epoch = 0

if os.path.exists(checkpoint_dir):
    # Get all checkpoint directories (e.g., 'step_200', 'step_400')
    step_dirs = [d for d in os.listdir(checkpoint_dir) if d.startswith('step_')]

    if step_dirs:
        # Find the step number from each directory name and get the max
        latest_step = max([int(re.search(r'\d+', d).group()) for d in step_dirs])
        latest_checkpoint_dir = os.path.join(checkpoint_dir, f"step_{latest_step}")

# --- LOAD FROM CHECKPOINT IF ONE WAS FOUND ---
if latest_checkpoint_dir:
    print(f"✅ Resuming training from the latest checkpoint: {latest_checkpoint_dir}")

    # Load the LoRA adapter weights
    # strict=False is important as we are only loading the adapter, not the whole model
    adapter_weights_path = os.path.join(latest_checkpoint_dir, "adapter_model.safetensors")
    state_dict = load_file(adapter_weights_path)
    model.load_state_dict(state_dict, strict=False)
    print("✅ Model adapter weights loaded successfully.")

    # 2. Check if the current optimizer is the one we want to use (AdamW8bit)
    if isinstance(optimizer, bnb.optim.AdamW8bit):
        try:
            # Try to load the full checkpoint, including optimizer/scaler
            checkpoint = torch.load(os.path.join(latest_checkpoint_dir, "optimizer.pt"))
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
            scaler.load_state_dict(checkpoint['scaler_state_dict'])
            print("✅ Optimizer, scheduler, and scaler states loaded successfully.")
        except Exception as e:
            # This will trigger on the first run after switching to AdamW8bit
            print(f"⚠️ Could not load optimizer/scaler state. Starting fresh. This is expected.")
            print(f"   Error details: {e}")
    else:
        # If we're not using an 8-bit optimizer, just try to load optimizer/scheduler
        try:
            checkpoint = torch.load(os.path.join(latest_checkpoint_dir, "optimizer.pt"))
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
            print("✅ Optimizer and scheduler states loaded successfully (non-8bit).")
        except Exception as e:
            print(f"⚠️ Could not load optimizer/scheduler state (non-8bit). Starting fresh. Error: {e}")

    # Restore the global step to continue numbering correctly
    # We can parse it from the directory name for simplicity
    global_step = latest_step
    start_epoch = global_step * GRADIENT_ACCUM // len(loader)
    print(f"Resuming at global_step={global_step} (Epoch ~{start_epoch+1})")

else:
    print("No checkpoint found. Starting training from scratch.")

model.train()

for epoch in range(start_epoch, EPOCHS):
    print(f"\n=== Epoch {epoch + 1}/{EPOCHS} ===\n")

    if epoch == start_epoch:
        steps_in_epoch_done = (global_step * GRADIENT_ACCUM) % len(loader)
    else:
        steps_in_epoch_done = 0

    pbar = tqdm(enumerate(loader), total=len(loader), desc=f"Epoch {epoch + 1}")

    for step, batch in pbar:

        if step < steps_in_epoch_done:
            continue

        # Move batch to device
        batch = {k: v.to(DEVICE) for k, v in batch.items()}
        outputs = model(**batch, use_cache=False)
        loss = outputs.loss / GRADIENT_ACCUM
        scaler.scale(loss).backward()

        # Gradient accumulation
        if (step + 1) % GRADIENT_ACCUM == 0:
            # Gradient clipping
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)

            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            scheduler.step()

            global_step += 1

            if global_step % 10 == 0:
                running_loss = 0.0
                print(f"Step {global_step}, Avg Loss: {loss.item() * GRADIENT_ACCUM:.4f}")

            #Checkpoint saving logic
            if global_step % SAVE_EVERY_N_STEPS == 0:
                print(f"\nSaving checkpoint at step {global_step}...")
                step_checkpoint_dir = os.path.join(checkpoint_dir, f"step_{global_step}")
                os.makedirs(step_checkpoint_dir, exist_ok=True)

                # Save the PEFT model adapters
                model.save_pretrained(step_checkpoint_dir)

                # Save optimizer and other states
                torch.save({
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),
                    'scaler_state_dict': scaler.state_dict()
                }, os.path.join(step_checkpoint_dir, "optimizer.pt"))

                print(f"Checkpoint saved to {step_checkpoint_dir}\n")

print("Training finished.")

print(f"\nSaving final trained model to {final_model_dir}...")

model.save_pretrained(final_model_dir)

print("Final model saved successfully.")


import shutil
import os

print("📦 Zipping final model for download...")

# Output filename
zip_name = "mind2web_trained_model.zip"
zip_path = f"/workspace/{zip_name}"

# Create the zip file
# format: zip, root_dir: folder to zip
shutil.make_archive(zip_path.replace('.zip', ''), 'zip', final_model_dir)

print(f"✅ Model zipped successfully: {zip_path}")
print(f"Size: {os.path.getsize(zip_path) / (1024*1024*1024):.2f} GB")
print("\nTo download this to your Mac, run the command below on YOUR LOCAL TERMINAL:")
print(f"scp -P [PORT] root@[IP_ADDRESS]:{zip_path} ~/Downloads/")


No checkpoint found. Starting training from scratch.

=== Epoch 1/5 ===



Epoch 1:   0%|          | 0/5743 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Epoch 1:   1%|▏         | 80/5743 [08:20<9:40:12,  6.15s/it] 

Step 10, Avg Loss: 11.8598


Epoch 1:   3%|▎         | 160/5743 [16:05<8:33:40,  5.52s/it] 

Step 20, Avg Loss: 12.6567


...

Step 3570, Avg Loss: 3.5725


Epoch 5:  99%|█████████▉| 5696/5743 [9:42:52<04:45,  6.07s/it]

Step 3580, Avg Loss: 2.7643


Epoch 5: 100%|██████████| 5743/5743 [9:47:27<00:00,  6.14s/it]


Training finished.

Saving final trained model to /workspace/mind2web_trained_model...
Final model saved successfully.
📦 Zipping final model for download...
✅ Model zipped successfully: /workspace/mind2web_trained_model.zip
Size: 0.08 GB

To download this to your Mac, run the command below on YOUR LOCAL TERMINAL:
scp -P [PORT] root@[IP_ADDRESS]:/workspace/mind2web_trained_model.zip ~/Downloads/


In [ ]:
import shutil
import os

# Define the source paths
base_path = "/workspace"
checkpoints_src = os.path.join(base_path, "qwen_checkpoints")
model_src = os.path.join(base_path, "mind2web_trained_model")

# Define the output names
zip_1 = os.path.join(base_path, "all_checkpoints") # becomes all_checkpoints.zip
zip_2 = os.path.join(base_path, "final_model")     # becomes final_model.zip

print("📦 Zipping Checkpoints (This includes all steps, so it will be large)...")
# format='zip', root_dir=folder_to_compress
shutil.make_archive(zip_1, 'zip', checkpoints_src)
print(f"✅ Created: {zip_1}.zip")

print("\n📦 Zipping Final Model...")
shutil.make_archive(zip_2, 'zip', model_src)
print(f"✅ Created: {zip_2}.zip")

# Print sizes so you know what to expect
size_1 = os.path.getsize(f"{zip_1}.zip") / (1024*1024*1024)
size_2 = os.path.getsize(f"{zip_2}.zip") / (1024*1024*1024)

print(f"\n📊 File Sizes:")
print(f"Checkpoints: {size_1:.2f} GB")
print(f"Final Model: {size_2:.2f} GB")

📦 Zipping Checkpoints (This includes all steps, so it will be large)...
✅ Created: /workspace/all_checkpoints.zip

📦 Zipping Final Model...
✅ Created: /workspace/final_model.zip

📊 File Sizes:
Checkpoints: 4.76 GB
Final Model: 0.08 GB


# Quick inference

In [ ]:
import torch
from transformers import AutoProcessor, Qwen2_5_VLForConditionalGeneration, BitsAndBytesConfig
from peft import PeftModel
from PIL import Image
import json
import os

# 1. Define Paths
# If on Colab, upload your unzipped 'final_model' folder to /content/final_model
# If on RunPod, it's at /workspace/mind2web_trained_model
ADAPTER_PATH = "/workspace/mind2web_trained_model" # Update this path!

# 2. Load Base Model (4-bit to save memory)
model_id = "Qwen/Qwen2.5-VL-7B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

print("⏳ Loading Base Model...")
base_model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
    low_cpu_mem_usage=True
)

print("⏳ Loading Processor...")
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

# 3. Load Your Fine-Tuned LoRA Adapter
print("🔗 Attaching LoRA Adapter...")
model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)
model.eval() # Set to inference mode

print("✅ Model Ready for Inference!")

⏳ Loading Base Model...


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

⏳ Loading Processor...


The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


🔗 Attaching LoRA Adapter...
✅ Model Ready for Inference!


In [ ]:
def run_inference(sample, image_path):
    # 1. Prepare Image (Apply the same Resize/Crop logic as training)
    raw_image = Image.open(image_path).convert("RGB")

    # Resize Logic (Match your training script)
    TARGET_WIDTH = 1024
    MAX_HEIGHT = 1280
    w_percent = (TARGET_WIDTH / float(raw_image.size[0]))
    h_size = int((float(raw_image.size[1]) * float(w_percent)))

    img_resized = raw_image.resize((TARGET_WIDTH, h_size), Image.LANCZOS)
    if h_size > MAX_HEIGHT:
        img_resized = img_resized.crop((0, 0, TARGET_WIDTH, MAX_HEIGHT))

    # 2. Prepare Prompt
    # We strip the "System" prompt structure because apply_chat_template handles it
    # We just need the instruction text.

    # Extract the raw instruction from your processed prompt
    # Your prompt stored in JSONL usually looks like: "You are a web agent...\nTASK:..."
    # We can pass that directly.
    prompt_text = sample['prompt']

    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": img_resized},
                {"type": "text", "text": prompt_text},
            ],
        }
    ]

    # 3. Process Inputs
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

    inputs = processor(
        text=[text],
        images=[img_resized],
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")

    # 4. Generate
    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_new_tokens=128,  # JSON is usually short
            do_sample=False,     # Greedy decoding (deterministic, best for code/json)
            temperature=0.0      # Strict adherence
        )

    # 5. Decode
    # Determine where the input ends and output begins
    input_len = inputs.input_ids.shape[1]
    generated_ids_trimmed = [
        out_ids[input_len:] for out_ids in generated_ids
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )

    return output_text[0]

In [ ]:
import json
import random

# Path to your processed data
JSONL_PATH = "/workspace/mind2web_processed_train.jsonl" # Update if needed
# Path to your local images
IMG_FOLDER = "/workspace/m2w_i_local" # Update if needed

print("🔍 Loading examples...")
examples = []
with open(JSONL_PATH, 'r') as f:
    for line in f:
        examples.append(json.loads(line))

# Pick 3 random examples
test_samples = random.sample(examples, 3)

for i, sample in enumerate(test_samples):
    print(f"\n================ EXAMPLE {i+1} ================")

    # Construct image path
    img_filename = f"{sample['annotation_id']}.jpeg"
    img_path = os.path.join(IMG_FOLDER, img_filename)

    if not os.path.exists(img_path):
        print(f"⚠️ Image missing: {img_path}")
        continue

    print(f"🎯 TASK: {sample['prompt'].split('TASK:')[1].split('ELEMENTS:')[0].strip()}")
    print(f"🖼️ Image: {img_filename}")

    try:
        # Run Inference
        prediction_str = run_inference(sample, img_path)

        print("\n🤖 MODEL PREDICTION:")
        print(prediction_str)

        print("\n✅ GROUND TRUTH:")
        print(sample['label'])

    except Exception as e:
        print(f"❌ Error: {e}")

🔍 Loading examples...

================ EXAMPLE 1 ================
🎯 TASK: View details for a Times Square parking lot that is wheelchair accessible.
🖼️ Image: 6b627cbc-a45e-4f7e-9d02-bfca1a41070a.jpeg

🤖 MODEL PREDICTION:
{"action": "click", "element_id": "2149", "value": "", "is_finished": false}

✅ GROUND TRUTH:
{"action": "click", "element_id": "2149", "value": "", "is_finished": false}

================ EXAMPLE 2 ================
🎯 TASK: Find a parking lot in Gloucester and book a ride from there to North Plymouth on April 28, 2:30 pm, view the map to understand the route better.
🖼️ Image: 504c0c6b-7e78-4bfa-ae3f-00f8e59c3693.jpeg

🤖 MODEL PREDICTION:
{"action": "click", "element_id": "101371", "value": "", "is_finished": false}

✅ GROUND TRUTH:
{"action": "select", "element_id": "101373", "value": "PM", "is_finished": false}

================ EXAMPLE 3 ================
🎯 TASK: Add 3 movies from At the Kiosk which are new to Wishlist.
🖼️ Image: c5871c3f-27c3-48d8-bdf9-8b49382a2719